In [1]:
import pandas as pd
import numpy as np
import funcoes_importantes as fi
fp = fi.funcoes_main()

In [8]:
# Antes de tudos normalizar z-score os dados com as info do dados de treino(mean e std)
# iSSO TUDO PARA 6 CLASSES
# Antes de tudo diagonalizar a mcov 

def nayves_bayes(X,y):
    # 1- calcula a probilidade a priori[1x1]
    probs_p = fp.prob_a_priori(X)

    # 2- vetor médio por classe[1x34]
    vt_medio_1 = fp.vetor_medio(X.query('c35==1'))
    vt_medio_2 = fp.vetor_medio(X.query('c35==2'))
    vt_medio_3 = fp.vetor_medio(X.query('c35==3'))
    vt_medio_4 = fp.vetor_medio(X.query('c35==4'))
    vt_medio_5 = fp.vetor_medio(X.query('c35==5'))
    vt_medio_6 = fp.vetor_medio(X.query('c35==6'))
    
    mcov_1 = fp.matriz_covariancia(X.query('c35==1'))
    mcov_2 = fp.matriz_covariancia(X.query('c35==2'))
    mcov_3 = fp.matriz_covariancia(X.query('c35==3'))
    mcov_4 = fp.matriz_covariancia(X.query('c35==4'))
    mcov_5 = fp.matriz_covariancia(X.query('c35==5'))
    mcov_6 = fp.matriz_covariancia(X.query('c35==6'))

    # 3- matriz covariancia por classe[34x34] sabendo que são descorrelacionados
    mcov_1_descorrelacionada = fp.limpar_covariancia(mcov_1)
    mcov_2_descorrelacionada = fp.limpar_covariancia(mcov_2)
    mcov_3_descorrelacionada = fp.limpar_covariancia(mcov_3)
    mcov_4_descorrelacionada = fp.limpar_covariancia(mcov_4)
    mcov_5_descorrelacionada = fp.limpar_covariancia(mcov_5)
    mcov_6_descorrelacionada = fp.limpar_covariancia(mcov_6)

    # 4- Fazer a correção lamdba A = A + xI na matriz de covariancia
    mcov_1_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_2_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_3_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_4_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_5_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01
    mcov_6_lambda_d = mcov_1_descorrelacionada + np.identity(mcov_1_descorrelacionada.shape[0], dtype=float)*0.01

    # 5- Calcular a inversa da matriz de covairância regulada lambda
    mcov_1_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_1_lambda_d.values, dtype='float')))
    mcov_2_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_2_lambda_d.values, dtype='float')))
    mcov_3_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_3_lambda_d.values, dtype='float')))
    mcov_4_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_4_lambda_d.values, dtype='float')))
    mcov_5_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_5_lambda_d.values, dtype='float')))
    mcov_6_lambda_inv_d = pd.DataFrame(np.linalg.inv(np.matrix(mcov_6_lambda_d.values, dtype='float')))
    
    # 6- Calcular o determinante de cada matriz de covariância regulada
    det_1 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_2 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_3 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_4 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_5 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    det_6 = np.linalg.det(np.matrix(mcov_1_lambda_inv_d.values, dtype='float'))
    
    
    # 5- Teste, pegar agora um elementro da base de teste e aplicar
    # min: ln|det(mcov_classe_i)| + 
    #       (elem_teste-vetor_medio_classe_i).T*mcov_classe_i_inv*(elem_teste-vetor_md_classe_i)
    #       - 2*ln(P_priori(classe_i)) 
    aux = []
    for i in y.index.values:
        valores = []
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_1,mcov_1_lambda_inv_d,det_1,probs_p[0]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_2,mcov_2_lambda_inv_d,det_2,probs_p[1]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_3,mcov_3_lambda_inv_d,det_3,probs_p[2]))    
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_4,mcov_4_lambda_inv_d,det_4,probs_p[3]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_5,mcov_5_lambda_inv_d,det_5,probs_p[4]))
        valores.append(fp.teste_elem(y.iloc[i,:-1],vt_medio_6,mcov_6_lambda_inv_d,det_6,probs_p[5]))
        aux.append(valores.index(min(valores))+1)
    print('Acrurácia de ',fp.get_acc(aux,y['c35'])*100,'%')
    return fp.get_acc(aux,y['c35'])

In [9]:
X,y = fp.kfold_shuffle()
nayves_bayes(X,y)

ValueError: Dot product shape mismatch, (34, 34) vs (35, 34)

In [13]:
# nayves_bayes_accs = []
# for _ in range(50):
#     X,y = fi.kfold_shuffle(derma_dados,5)
#     nayves_bayes_accs.append(nayves_bayes(X,y))
# print('Acurácia média de ',round(np.mean(nayves_bayes_accs),4)*100,'%')